In [1]:
import pandas as pd
path = 'https://raw.githubusercontent.com/cinnData/MLearning/main/Data/'
df = pd.read_csv(path + 'spam.csv')

In [2]:
df.spam.value_counts()

0    2788
1    1813
Name: spam, dtype: int64